In [ ]:
# print("""================== Loading Train Features and Testing Features =============""")

# import pandas as pd
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
# import pickle

# """=================== Getting embedding for test file ==============="""
# import pandas as pd
# import numpy as np
# import re
# import nltk
# import torch
# import pickle
# from collections import Counter
# from nltk.corpus import stopwords
# from transformers import BertTokenizer, BertModel
# from sklearn.metrics.pairwise import cosine_similarity

# # Ensure NLTK's stopwords are downloaded
# nltk.download('stopwords')

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# train = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet')


# # Load BERT model and tokenizer, and transfer the model to GPU if available
# print("Loading BERT model...")
# bert_tokenizer = BertTokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_tokenizer')
# bert_model = BertModel.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_model').to(device)


# # Text preprocessing function
# def preprocess_text(text):
#     if isinstance(text, str):  # Ensure text is a string before processing
#         text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
#         words = text.lower().split()  # Lowercase and split the text into words
#         stop_words = set(stopwords.words('english'))  # Set of stopwords
#         words = [word for word in words if word not in stop_words]  # Remove stopwords
#         unique_words = list(Counter(words).keys())  # Get unique words
#         return ' '.join(unique_words)  # Join them back into a string
#     return ''  # Return empty string if text is not a valid string


# # Apply preprocessing only to specific columns
# columns_to_preprocess = ['prompt', 'response_a', 'response_b']  # Replace with actual column names
# for col in columns_to_preprocess:
#     train[col] = train[col].apply(preprocess_text)
#     # test[col] = test[col].apply(preprocess_text)


# # Function to extract embeddings from BERT
# def extract_embeddings(text, model, tokenizer, max_length=512):
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mean pooling over token embeddings
#     return embedding


# # Function to save embeddings to a pickle file
# def save_embeddings(data, filename):
#     with open(filename, 'wb') as f:
#         pickle.dump(data, f)
#     print(f"Saved embeddings to {filename}")


# # Function to generate and save embeddings for each column
# def generate_and_save_embeddings(df, column_name):
#     print(f"Processing column: {column_name}")
#     column_data = df[column_name].dropna()  # Remove any NaN values if present

#     # BERT embeddings
#     embeddings = np.array([extract_embeddings(text, bert_model, bert_tokenizer) for text in column_data])
#     save_embeddings(embeddings, f'/kaggle/working/{column_name}_bert.pkl')
#     print(f"SAVED IS  {column_name}")
#     return embeddings


# # Generate and save embeddings for each column
# prompt_embeddings = generate_and_save_embeddings(train, 'prompt')
# response_a_embeddings = generate_and_save_embeddings(train, 'response_a')
# response_b_embeddings = generate_and_save_embeddings(train, 'response_b')


# # Define function to calculate metrics
# def calculate_metrics(prompt_emb, response_a_emb, response_b_emb):
#     metrics = {}

#     # 1. Cosine Similarity
#     metrics['cosine_similarity_a'] = cosine_similarity([prompt_emb], [response_a_emb])[0][0]
#     metrics['cosine_similarity_b'] = cosine_similarity([prompt_emb], [response_b_emb])[0][0]

#     # 2. Cosine Distance
#     metrics['cosine_distance_a'] = 1 - metrics['cosine_similarity_a']
#     metrics['cosine_distance_b'] = 1 - metrics['cosine_similarity_b']

#     # 3. Mean Euclidean Distance
#     metrics['euclidean_distance_a'] = euclidean_distances([prompt_emb], [response_a_emb])[0][0]
#     metrics['euclidean_distance_b'] = euclidean_distances([prompt_emb], [response_b_emb])[0][0]

#     # 4. Semantic Overlap (measured by cosine similarity)
#     metrics['semantic_overlap_a'] = metrics['cosine_similarity_a']
#     metrics['semantic_overlap_b'] = metrics['cosine_similarity_b']

#     # 5. Topic Coherence (same as semantic overlap for now)
#     metrics['topic_coherence_a'] = metrics['cosine_similarity_a']
#     metrics['topic_coherence_b'] = metrics['cosine_similarity_b']

#     # 6. Prompt Adherence
#     metrics['prompt_adherence_a'] = metrics['cosine_similarity_a']
#     metrics['prompt_adherence_b'] = metrics['cosine_similarity_b']

#     # 7. Contextual Consistency
#     metrics['contextual_consistency'] = cosine_similarity([response_a_emb], [response_b_emb])[0][0]

#     # 8. Diversity of Response
#     metrics['response_diversity'] = 1 - metrics['contextual_consistency']

#     # 9. Sentiment Similarity Score
#     metrics['sentiment_similarity_a'] = metrics['cosine_similarity_a']  # Proxy
#     metrics['sentiment_similarity_b'] = metrics['cosine_similarity_b']  # Proxy

#     # 10. Topic Coverage (same as semantic overlap for simplicity)
#     metrics['topic_coverage_a'] = metrics['cosine_similarity_a']
#     metrics['topic_coverage_b'] = metrics['cosine_similarity_b']


#     # 12. Depth of Response (Variance of embeddings as proxy)
#     metrics['depth_of_response_a'] = np.var(response_a_emb)
#     metrics['depth_of_response_b'] = np.var(response_b_emb)

#     # 13. Focus Score (Inverse of Euclidean Distance)
#     metrics['focus_score_a'] = 1 / (1 + metrics['euclidean_distance_a'])
#     metrics['focus_score_b'] = 1 / (1 + metrics['euclidean_distance_b'])

#     # 14. Embedding Vector Magnitude Ratio
#     metrics['embedding_magnitude_ratio_a'] = np.linalg.norm(response_a_emb) / np.linalg.norm(prompt_emb)
#     metrics['embedding_magnitude_ratio_b'] = np.linalg.norm(response_b_emb) / np.linalg.norm(prompt_emb)

#     # 15. Off-Topic Score
#     metrics['off_topic_a'] = metrics['cosine_distance_a']
#     metrics['off_topic_b'] = metrics['cosine_distance_b']


#     # 16. Euclidean Distance
#     metrics['euclidean_distance_a'] = euclidean_distances([prompt_emb], [response_a_emb])[0][0]
#     metrics['euclidean_distance_b'] = euclidean_distances([prompt_emb], [response_b_emb])[0][0]

#     # 17. Embedding Density
#     metrics['embedding_density_a'] = np.sum(np.square(response_a_emb))
#     metrics['embedding_density_b'] = np.sum(np.square(response_b_emb))

#     # 18. Internal Coherence
#     metrics['internal_coherence_a'] = np.mean(cosine_similarity([response_a_emb]))
#     metrics['internal_coherence_b'] = np.mean(cosine_similarity([response_b_emb]))

#     # 19. Response Concreteness
#     metrics['response_concreteness_a'] = np.std(response_a_emb)
#     metrics['response_concreteness_b'] = np.std(response_b_emb)

#     # 20. Redundancy Score
#     metrics['redundancy_score_a'] = 1 - np.mean(np.std(response_a_emb))
#     metrics['redundancy_score_b'] = 1 - np.mean(np.std(response_b_emb))

#     # 21. Semantic Entailment (similar to cosine similarity)
#     metrics['semantic_entailment_a'] = metrics['cosine_similarity_a']
#     metrics['semantic_entailment_b'] = metrics['cosine_similarity_b']

#     # 22. Prompt-Response Embedding Ratio
#     metrics['embedding_ratio_a'] = np.linalg.norm(response_a_emb) / np.linalg.norm(prompt_emb)
#     metrics['embedding_ratio_b'] = np.linalg.norm(response_b_emb) / np.linalg.norm(prompt_emb)

#     # 23. Informativeness Score
#     metrics['informativeness_a'] = np.var(response_a_emb)
#     metrics['informativeness_b'] = np.var(response_b_emb)

#     # 24. Topic Drift Score
#     metrics['topic_drift_a'] = np.std(response_a_emb - prompt_emb)
#     metrics['topic_drift_b'] = np.std(response_b_emb - prompt_emb)

#     # 25. Context Alignment
#     metrics['context_alignment'] = cosine_similarity([response_a_emb], [response_b_emb])[0][0]

#     #26. Disparity Score
#     metrics['disparity_score'] = euclidean_distances([response_a_emb], [response_b_emb])[0][0]


#     return metrics


# def load_data(prompt_path,response_a_path , response_b_path):
#     # Load precomputed embeddings (assuming they've been generated and saved as instructed)
#     prompt_embeddings = pickle.load(open(prompt_path, 'rb'))
#     response_a_embeddings = pickle.load(open(response_a_path, 'rb'))
#     response_b_embeddings = pickle.load(open(response_b_path , 'rb'))

#     train_features_data = []
#     for i in range(len(prompt_embeddings)):
#         metrics = calculate_metrics(prompt_embeddings[i], response_a_embeddings[i], response_b_embeddings[i])
#         train_features_data.append(metrics)
#     train_features_data = pd.DataFrame(train_features_data)
#     return train_features_data

# prompt_path = '/kaggle/input/bert-embedding-of-train-data/prompt_bert.pkl'
# response_a_path = '/kaggle/input/bert-embedding-of-train-data/response_a_bert.pkl'
# response_b_path  =  '/kaggle/input/bert-embedding-of-train-data/response_b_bert.pkl'
# train_features_from_embedding = load_data(prompt_path,response_a_path , response_b_path)

# prompt_path = '/kaggle/working/prompt_bert.pkl'
# response_a_path ='/kaggle/working/response_a_bert.pkl'
# response_b_path = '/kaggle/working/response_b_bert.pkl'
# test_features_from_embedding = load_data(prompt_path,response_a_path , response_b_path)

# print('======================== TRAIN_FEATURES AND TEST_FEATURES LOADED ===============================')





In [ ]:
print("""================== Loading Train Features and Testing Features =============""")

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import pickle

"""=================== Getting embedding for test file ==============="""
import pandas as pd
import numpy as np
import re
import nltk
import torch
import pickle
from collections import Counter
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Ensure NLTK's stopwords are downloaded
nltk.download('stopwords')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')
# train = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')


# Load BERT model and tokenizer, and transfer the model to GPU if available
print("Loading BERT model...")
bert_tokenizer = BertTokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_tokenizer')
bert_model = BertModel.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_model').to(device)


# Text preprocessing function
def preprocess_text(text):
    if isinstance(text, str):  # Ensure text is a string before processing
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        words = text.lower().split()  # Lowercase and split the text into words
        stop_words = set(stopwords.words('english'))  # Set of stopwords
        words = [word for word in words if word not in stop_words]  # Remove stopwords
        unique_words = list(Counter(words).keys())  # Get unique words
        return ' '.join(unique_words)  # Join them back into a string
    return ''  # Return empty string if text is not a valid string


# Apply preprocessing only to specific columns
columns_to_preprocess = ['prompt', 'response_a', 'response_b']  # Replace with actual column names
for col in columns_to_preprocess:
    # train[col] = train[col].apply(preprocess_text)
    test[col] = test[col].apply(preprocess_text)


# Function to extract embeddings from BERT
def extract_embeddings(text, model, tokenizer, max_length=512):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mean pooling over token embeddings
    return embedding


# Function to save embeddings to a pickle file
def save_embeddings(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Saved embeddings to {filename}")


# Function to generate and save embeddings for each column
def generate_and_save_embeddings(df, column_name):
    print(f"Processing column: {column_name}")
    column_data = df[column_name].dropna()  # Remove any NaN values if present

    # BERT embeddings
    embeddings = np.array([extract_embeddings(text, bert_model, bert_tokenizer) for text in column_data])
    save_embeddings(embeddings, f'/kaggle/working/{column_name}_bert.pkl')
    print(f"SAVED IS  {column_name}")
    return embeddings


# Generate and save embeddings for each column
# prompt_embeddings = generate_and_save_embeddings(train, 'prompt')
# response_a_embeddings = generate_and_save_embeddings(train, 'response_a')
# response_b_embeddings = generate_and_save_embeddings(train, 'response_b')

prompt_embeddings = generate_and_save_embeddings(test, 'prompt')
response_a_embeddings = generate_and_save_embeddings(test, 'response_a')
response_b_embeddings = generate_and_save_embeddings(test, 'response_b')



# Define function to calculate metrics
def calculate_metrics(prompt_emb, response_a_emb, response_b_emb):
    metrics = {}

    # 1. Cosine Similarity
    metrics['cosine_similarity_a'] = cosine_similarity([prompt_emb], [response_a_emb])[0][0]
    metrics['cosine_similarity_b'] = cosine_similarity([prompt_emb], [response_b_emb])[0][0]

    # 2. Cosine Distance
    metrics['cosine_distance_a'] = 1 - metrics['cosine_similarity_a']
    metrics['cosine_distance_b'] = 1 - metrics['cosine_similarity_b']

    # 3. Mean Euclidean Distance
    metrics['euclidean_distance_a'] = euclidean_distances([prompt_emb], [response_a_emb])[0][0]
    metrics['euclidean_distance_b'] = euclidean_distances([prompt_emb], [response_b_emb])[0][0]

    # 4. Semantic Overlap (measured by cosine similarity)
    metrics['semantic_overlap_a'] = metrics['cosine_similarity_a']
    metrics['semantic_overlap_b'] = metrics['cosine_similarity_b']

    # 5. Topic Coherence (same as semantic overlap for now)
    metrics['topic_coherence_a'] = metrics['cosine_similarity_a']
    metrics['topic_coherence_b'] = metrics['cosine_similarity_b']

    # 6. Prompt Adherence
    metrics['prompt_adherence_a'] = metrics['cosine_similarity_a']
    metrics['prompt_adherence_b'] = metrics['cosine_similarity_b']

    # 7. Contextual Consistency
    metrics['contextual_consistency'] = cosine_similarity([response_a_emb], [response_b_emb])[0][0]

    # 8. Diversity of Response
    metrics['response_diversity'] = 1 - metrics['contextual_consistency']

    # 9. Sentiment Similarity Score
    metrics['sentiment_similarity_a'] = metrics['cosine_similarity_a']  # Proxy
    metrics['sentiment_similarity_b'] = metrics['cosine_similarity_b']  # Proxy

    # 10. Topic Coverage (same as semantic overlap for simplicity)
    metrics['topic_coverage_a'] = metrics['cosine_similarity_a']
    metrics['topic_coverage_b'] = metrics['cosine_similarity_b']


    # 12. Depth of Response (Variance of embeddings as proxy)
    metrics['depth_of_response_a'] = np.var(response_a_emb)
    metrics['depth_of_response_b'] = np.var(response_b_emb)

    # 13. Focus Score (Inverse of Euclidean Distance)
    metrics['focus_score_a'] = 1 / (1 + metrics['euclidean_distance_a'])
    metrics['focus_score_b'] = 1 / (1 + metrics['euclidean_distance_b'])

    # 14. Embedding Vector Magnitude Ratio
    metrics['embedding_magnitude_ratio_a'] = np.linalg.norm(response_a_emb) / np.linalg.norm(prompt_emb)
    metrics['embedding_magnitude_ratio_b'] = np.linalg.norm(response_b_emb) / np.linalg.norm(prompt_emb)

    # 15. Off-Topic Score
    metrics['off_topic_a'] = metrics['cosine_distance_a']
    metrics['off_topic_b'] = metrics['cosine_distance_b']


    # 16. Euclidean Distance
    metrics['euclidean_distance_a'] = euclidean_distances([prompt_emb], [response_a_emb])[0][0]
    metrics['euclidean_distance_b'] = euclidean_distances([prompt_emb], [response_b_emb])[0][0]

    # 17. Embedding Density
    metrics['embedding_density_a'] = np.sum(np.square(response_a_emb))
    metrics['embedding_density_b'] = np.sum(np.square(response_b_emb))

    # 18. Internal Coherence
    metrics['internal_coherence_a'] = np.mean(cosine_similarity([response_a_emb]))
    metrics['internal_coherence_b'] = np.mean(cosine_similarity([response_b_emb]))

    # 19. Response Concreteness
    metrics['response_concreteness_a'] = np.std(response_a_emb)
    metrics['response_concreteness_b'] = np.std(response_b_emb)

    # 20. Redundancy Score
    metrics['redundancy_score_a'] = 1 - np.mean(np.std(response_a_emb))
    metrics['redundancy_score_b'] = 1 - np.mean(np.std(response_b_emb))

    # 21. Semantic Entailment (similar to cosine similarity)
    metrics['semantic_entailment_a'] = metrics['cosine_similarity_a']
    metrics['semantic_entailment_b'] = metrics['cosine_similarity_b']

    # 22. Prompt-Response Embedding Ratio
    metrics['embedding_ratio_a'] = np.linalg.norm(response_a_emb) / np.linalg.norm(prompt_emb)
    metrics['embedding_ratio_b'] = np.linalg.norm(response_b_emb) / np.linalg.norm(prompt_emb)

    # 23. Informativeness Score
    metrics['informativeness_a'] = np.var(response_a_emb)
    metrics['informativeness_b'] = np.var(response_b_emb)

    # 24. Topic Drift Score
    metrics['topic_drift_a'] = np.std(response_a_emb - prompt_emb)
    metrics['topic_drift_b'] = np.std(response_b_emb - prompt_emb)

    # 25. Context Alignment
    metrics['context_alignment'] = cosine_similarity([response_a_emb], [response_b_emb])[0][0]

    #26. Disparity Score
    metrics['disparity_score'] = euclidean_distances([response_a_emb], [response_b_emb])[0][0]


    return metrics


def load_data(prompt_path,response_a_path , response_b_path):
    # Load precomputed embeddings (assuming they've been generated and saved as instructed)
    prompt_embeddings = pickle.load(open(prompt_path, 'rb'))
    response_a_embeddings = pickle.load(open(response_a_path, 'rb'))
    response_b_embeddings = pickle.load(open(response_b_path , 'rb'))

    train_features_data = []
    for i in range(len(prompt_embeddings)):
        metrics = calculate_metrics(prompt_embeddings[i], response_a_embeddings[i], response_b_embeddings[i])
        train_features_data.append(metrics)
    train_features_data = pd.DataFrame(train_features_data)
    return train_features_data

prompt_path = '/kaggle/input/training-llm-prompt-classifciation-bert-embedding/prompt_bert.pkl'
response_a_path = '/kaggle/input/training-llm-prompt-classifciation-bert-embedding/response_a_bert.pkl'
response_b_path  =  '/kaggle/input/training-llm-prompt-classifciation-bert-embedding/response_b_bert.pkl'
train_features_from_embedding = load_data(prompt_path,response_a_path , response_b_path)

prompt_path = '/kaggle/working/prompt_bert.pkl'
response_a_path ='/kaggle/working/response_a_bert.pkl'
response_b_path = '/kaggle/working/response_b_bert.pkl'
test_features_from_embedding = load_data(prompt_path,response_a_path , response_b_path)

print('======================== TRAIN_FEATURES AND TEST_FEATURES LOADED ===============================')



In [ ]:
print('=================== Training and Submission Creation =======')

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load training and testing data
train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

# Encode the target column into binary format
label_encoder = LabelEncoder()
train_df['target_encoded'] = label_encoder.fit_transform(train_df['winner'])  # Encodes `model_a` as 0, `model_b` as 1

# Define the features and target variable
y = train_df['target_encoded']


# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_features_from_embedding,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Initialize SVC model with a pipeline (scaling + classifier)
svc_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale data before passing to SVC
    ('svc', SVC(probability=False, random_state=42))  # `probability=False` for faster training
])

# Train the SVC model
print("Training SVC model...")
svc_pipeline.fit(X_train, y_train)
print("SVC training completed.")

# Validate the SVC model
print("Evaluating SVC model on validation set...")
svc_val_predictions = svc_pipeline.predict(X_val)
print("Classification Report for SVC:")
print(classification_report(y_val, svc_val_predictions, target_names=label_encoder.classes_))

# Make predictions on the test set using SVC
print("Making predictions on test data with SVC...")
svc_test_predictions = svc_pipeline.predict(test_features_from_embedding)

# Decode predictions back to the original labels
decoded_predictions = label_encoder.inverse_transform(svc_test_predictions)

# Create SVC submission DataFrame
svc_submission = pd.DataFrame({
    'id': test_df['id'], 
    'winner': decoded_predictions  # Add decoded predictions
})

# Save the SVC submission file
svc_submission_file = '/kaggle/working/submission.csv'
svc_submission.to_csv(svc_submission_file, index=False)
print(f"SVC submission saved to {svc_submission_file}")

# Optionally, display the first few rows of the SVC submission file
print("First few rows of the SVC submission file:")
print(svc_submission.head())


In [ ]:
# print("===========Getting Embedding(Vectors Tensor) of ============")

# import pandas as pd
# import numpy as np
# import re
# import nltk
# import torch
# import pickle
# from collections import Counter
# from nltk.corpus import stopwords
# from transformers import BertTokenizer, BertModel, XLNetTokenizer, XLNetModel, GPT2Tokenizer, GPT2Model, T5Tokenizer, T5Model
# import spacy

# # Ensure NLTK's stopwords are downloaded
# nltk.download('stopwords')

# # Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Load the data
# train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv', nrows=5000)
# test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv', nrows=5000)

# # Load models and transfer them to GPU if available
# print("Loading models...")

# # BERT
# bert_tokenizer = BertTokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_tokenizer')
# bert_model = BertModel.from_pretrained('/kaggle/input/vector-model-embedder/models/bert_model').to(device)


# # XLNet
# xlnet_tokenizer = XLNetTokenizer.from_pretrained("/kaggle/input/vector-model-embedder/models/xlnet_tokenizer")
# xlnet_model = XLNetModel.from_pretrained("/kaggle/input/vector-model-embedder/models/xlnet_model").to(device)


# # GPT-2
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/gpt2_tokenizer')
# gpt2_model = GPT2Model.from_pretrained('/kaggle/input/vector-model-embedder/models/gpt2_model').to(device)

# # T5
# t5_tokenizer = T5Tokenizer.from_pretrained('/kaggle/input/vector-model-embedder/models/t5_tokenizer')
# t5_model = T5Model.from_pretrained('/kaggle/input/vector-model-embedder/models/t5_model').to(device)


# # SpaCy
# nlp_local = spacy.load("/kaggle/input/vector-model-embedder/models/spacy_model")


# # Text preprocessing function
# def preprocess_text(text):
#     text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
#     words = text.lower().split()  # Lowercase and split the text into words
#     stop_words = set(stopwords.words('english'))  # Set of stopwords
#     words = [word for word in words if word not in stop_words]  # Remove stopwords
#     unique_words = list(Counter(words).keys())  # Get unique words
#     return ' '.join(unique_words)  # Join them back into a string

# # Apply preprocessing to all columns that will be used for embedding extraction
# train = train.applymap(preprocess_text)
# test = test.applymap(preprocess_text)

# # Function to extract embeddings from a model
# def extract_embeddings(text, model, tokenizer, max_length=512):
#     inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mean pooling over token embeddings
#     return embedding

# # Function to extract embeddings using SpaCy
# def text_to_vector(text):
#     doc = nlp_local(text)
#     return doc.vector

# # Function to save embeddings to pickle files
# def save_embeddings(data, filename):
#     with open(filename, 'wb') as f:
#         pickle.dump(data, f)
#     print(f"Saved embeddings to {filename}")

# # Function to generate and save embeddings for a list of columns
# def generate_and_save_embeddings(df, column_names):
#     for column_name in column_names:
#         print(f"Processing column: {column_name}")
#         column_data = df[column_name].dropna()  # Remove any NaN values if present
        
#         # BERT embeddings
#         bert_embeddings = np.array([extract_embeddings(text, bert_model, bert_tokenizer) for text in column_data])
#         save_embeddings(bert_embeddings, f'/kaggle/working/{column_name}_bert.pkl')

#         # XLNet embeddings
#         xlnet_embeddings = np.array([extract_embeddings(text, xlnet_model, xlnet_tokenizer) for text in column_data])
#         save_embeddings(xlnet_embeddings, f'/kaggle/working/{column_name}_xlnet.pkl')

#         # GPT-2 embeddings
#         gpt2_embeddings = np.array([extract_embeddings(text, gpt2_model, gpt2_tokenizer) for text in column_data])
#         save_embeddings(gpt2_embeddings, f'/kaggle/working/{column_name}_gpt2.pkl')

#         # T5 embeddings
#         t5_embeddings = np.array([extract_embeddings(text, t5_model, t5_tokenizer) for text in column_data])
#         save_embeddings(t5_embeddings, f'/kaggle/working/{column_name}_t5.pkl')

#         # SpaCy embeddings
#         spacy_embeddings = np.array([text_to_vector(text) for text in column_data])
#         save_embeddings(spacy_embeddings, f'/kaggle/working/{column_name}_spacy.pkl')

# # List of columns for which embeddings need to be generated
# columns_to_embed = ['prompt', 'response_a', 'response_b']  # Replace with actual column names

# # Generate and save embeddings for the specified columns in both train and test data
# generate_and_save_embeddings(train, columns_to_embed)
# generate_and_save_embeddings(test, columns_to_embed)

# print("All embeddings generated and saved successfully.")



In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, SGDClassifier, RidgeClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
# from sklearn.svm import SVC, LinearSVC, NuSVC
# from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
# from sklearn.ensemble import IsolationForest
# import xgboost as xgb
# from sklearn.metrics import accuracy_score

# path = "/kaggle/input/wsdm-cup-multilingual-chatbot-arena/"
# train = pd.read_parquet(path + "train.parquet")
# test = pd.read_parquet(path + "test.parquet")
# sub = pd.read_csv(path + "sample_submission.csv")

# # 10% as validation split, this percentage could be changed
# train, valid = train_test_split(train, test_size=0.2, stratify=train["winner"], random_state=161194)

# # Train set can be inverted (and winner too) to get twice the data from the available training dataset
# train_inv = train.copy()
# train_inv["response_a"], train_inv["response_b"] = train_inv["response_b"], train_inv["response_a"]
# train_inv["winner"] = train_inv["winner"].apply(lambda x: "model_a" if "b" in x else "model_b")

# # Compute features
# def compute_feats(df):
#     for col in ["response_a", "response_b", "prompt"]:
#         # response length is a key factor when choosing between two responses
#         df[f"{col}_len"] = df[f"{col}"].str.len()

#         # Some characters counting features 
#         df[f"{col}_spaces"] = df[f"{col}"].str.count("\s")
#         df[f"{col}_punct"] = df[f"{col}"].str.count(",|\.|!")
#         df[f"{col}_question_mark"] = df[f"{col}"].str.count("\?")
#         df[f"{col}_quot"] = df[f"{col}"].str.count("'|\"")
#         df[f"{col}_formatting_chars"] = df[f"{col}"].str.count("\*|\_")
#         df[f"{col}_math_chars"] = df[f"{col}"].str.count("\-|\+|\=")
#         df[f"{col}_curly_open"] = df[f"{col}"].str.count("\{")
#         df[f"{col}_curly_close"] = df[f"{col}"].str.count("}")
#         df[f"{col}_round_open"] = df[f"{col}"].str.count("\(")
#         df[f"{col}_round_close"] = df[f"{col}"].str.count("\)")
#         df[f"{col}_special_chars"] = df[f"{col}"].str.count("\W")
#         df[f"{col}_digits"] = df[f"{col}"].str.count("\d") > 0
#         df[f"{col}_letters"] = df[f"{col}"].str.count("[a-zA-Z]") > 0.1 * df[f"{col}_len"]
#         df[f"{col}_chinese"] = df[f"{col}"].str.count(r'[\u4e00-\u9fff]+') > 0.1 * df[f"{col}_len"]

#         # Features that show how balanced are curly and round brackets
#         df[f"{col}_round_balance"] = df[f"{col}_round_open"] - df[f"{col}_round_close"]
#         df[f"{col}_curly_balance"] = df[f"{col}_curly_open"] - df[f"{col}_curly_close"]

#         # Feature that tells if the string json is present somewhere
#         df[f"{col}_json"] = df[f"{col}"].str.lower().str.count("json")
#     return df

# train = compute_feats(train)
# train_inv = compute_feats(train_inv)
# train = pd.concat([train, train_inv])
# valid = compute_feats(valid)
# test = compute_feats(test)

# # Feature selection
# feats = list(train.columns)[8:]

# train["winner"] = (train["winner"] == "model_a").astype("int")
# valid["winner"] = (valid["winner"] == "model_a").astype("int")

# X = train[feats]
# y = train["winner"]

# X_val = valid[feats]
# y_val = valid["winner"]

# # List of classifiers to use
# classifiers = {
#     # "LogisticRegression": LogisticRegression(max_iter=1000),
#     # "SGDClassifier": SGDClassifier(),
#     "XGBoost": xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=6, objective='binary:logistic'),
#     # "MLPClassifier": MLPClassifier(max_iter=1000),
#     # "ExtraTreeClassifier": ExtraTreeClassifier(),
#     # "DecisionTreeClassifier": DecisionTreeClassifier(),
#     # "SVC": SVC(),
#     # "LinearSVC": LinearSVC(),
#     # "NuSVC": NuSVC(),
#     # "ExtraTreesClassifier": ExtraTreesClassifier(n_estimators=1000),
#     # "RandomForestClassifier": RandomForestClassifier(n_estimators=1000),
#     # "IsolationForest": IsolationForest(),
#     # "PassiveAggressiveClassifier": PassiveAggressiveClassifier(),
#     # "RidgeClassifier": RidgeClassifier()
# }

# # Train each model and generate predictions
# predictions = []

# for name, clf in classifiers.items():
#     print(f"Training {name}...")
#     clf.fit(X, y)
    
#     # Evaluate on validation set
#     y_pred_val = clf.predict(X_val)
#     print(f"{name} Accuracy on validation set: {accuracy_score(y_val, y_pred_val):.4f}")
    
#     # Predict on test set
#     y_pred_test = clf.predict(X_val)
#     test["winner"] = y_pred_test
#     test["winner"] = test["winner"].apply(lambda x: "model_a" if x == 1 else "model_b")
    
#     # Prepare submission
#     sub = test[["id", "winner"]]
#     sub.to_csv(f"/kaggle/working/submission.csv", index=False)
#     predictions.append(sub)

# # ensemble_predictions = np.mean([pred["winner"] == "model_a" for pred in predictions], axis=0)
# # final_submission = test.copy()
# # final_submission["winner"] = np.where(ensemble_predictions > 0.5, "model_a", "model_b")
# # final_submission.to_csv("submission_ensemble.csv", index=False)
